In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY,
        "X-Goog-Api-Key": GOOGLE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
	)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__
live = client.is_live()
connected = client.is_connected()

print(f"Weaviate Ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"Weaviate Live: {client.is_live()}")
print(f"Client Connected: {connected}")

In [ ]:
# Batching Snippet fixed_size recommeded method for production
col = client.collections.get("YourCollectionName")

try:
    with col.batch.fixed_size(batch_size=500, concurrent_requests=4) as batch:
        [YOUR_LOOP_FOR_DATA]
    failed_objs_a = col.batch.failed_objects  # Get failed objects
    if failed_objs_a:
        print(f"Number of failed objects in the first batch: {len(failed_objs_a)}")
        for i, failed_obj in enumerate(failed_objs_a, 1):
            print(f"Failed object {i}:")
            print(f"Error message: {failed_obj.message}")
    else:
        print("All objects were successfully added.")
except Exception as e:
    print(f"Error during batch import: {e}")
    print(f"Exception details: {str(e)}")

In [ ]:
# Batching Snippet with rate limiting method if Integration Models e.g. OpenAI/API is being used and Rate limits reached.
col = client.collections.get("YourCollectionName")

try:
    with col.batch.rate_limit(requests_per_minute=100) as batch:
        [YOUR_LOOP_FOR_DATA]
    failed_objs_a = col.batch.failed_objects  # Get failed objects
    if failed_objs_a:
        print(f"Number of failed objects in the first batch: {len(failed_objs_a)}")
        for i, failed_obj in enumerate(failed_objs_a, 1):
            print(f"Failed object {i}:")
            print(f"Error message: {failed_obj.message}")
    else:
        print("All objects were successfully added.")
except Exception as e:
    print(f"Error during batch import: {e}")
    print(f"Exception details: {str(e)}")

In [ ]:
# Batching Snippet with Dynamic, please use for none production or testing only.
col = client.collections.get("YourCollectionName")

try:
    with col.batch.dynamic() as batch:
        [YOUR_LOOP_FOR_DATA]
    failed_objs_a = col.batch.failed_objects  # Get failed objects
    if failed_objs_a:
        print(f"Number of failed objects in the first batch: {len(failed_objs_a)}")
        for i, failed_obj in enumerate(failed_objs_a, 1):
            print(f"Failed object {i}:")
            print(f"Error message: {failed_obj.message}")
    else:
        print("All objects were successfully added.")
except Exception as e:
    print(f"Error during batch import: {e}")
    print(f"Exception details: {str(e)}")

In [ ]:
# Batching Snippet with experimental, it's server-side batching mode 
# where it's decided by Weaviate server how to batch the data, 
# please use for none production or testing only for now until it's the recommended for production.
col = client.collections.get("YourCollectionName")

try:
    with col.batch.experimental() as batch:
        [YOUR_LOOP_FOR_DATA]
    failed_objs_a = col.batch.failed_objects  # Get failed objects
    if failed_objs_a:
        print(f"Number of failed objects in the first batch: {len(failed_objs_a)}")
        for i, failed_obj in enumerate(failed_objs_a, 1):
            print(f"Failed object {i}:")
            print(f"Error message: {failed_obj.message}")
    else:
        print("All objects were successfully added.")
except Exception as e:
    print(f"Error during batch import: {e}")
    print(f"Exception details: {str(e)}")

In [ ]:
# Generate 250 dummy records

dummy_data = []
for i in range(1, 251): 
    dummy_data.append({
        "title": f"Sample Item {i}",
        "description": f"This is a description for item number {i} in the dataset.",
        "value": i * 10.5,
        "status": "active" if i % 2 == 0 else "inactive",
        "timestamp": f"2024-{(i % 12) + 1:02d}-{(i % 28) + 1:02d}"
    })

# Batching Snippet with Tenants
col = client.collections.get("ShahinMultiTenantCollection")
tenant_name = "tenantA"  # Define your tenant name here

try:
    with col.with_tenant(tenant_name).batch.fixed_size(batch_size=500, concurrent_requests=4) as batch:
        for data_row in dummy_data:
            batch.add_object(
                properties=data_row,
            )
            
    failed_objs_a = col.batch.failed_objects  # Get failed objects
    if failed_objs_a:
        print(f"Number of failed objects: {len(failed_objs_a)}")
        for i, failed_obj in enumerate(failed_objs_a, 1):
            print(f"Failed object {i}: {failed_obj.message}")
    else:
        print("All objects were successfully added.")
except Exception as e:
    print(f"Error during batch import: {e}")